# SQL Guided Project: Analyzing CIA Factbook Data

This project will use data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/). It contains information about all countries, with variables such as area, population, and birth and death rates.

## Data overview

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db #read in the data

In [2]:
%%sql 
SELECT *
    FROM facts
    LIMIT 5
  ;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


Some additional information about the columns:
- `population_growth` shows the annual population growth rate as a percentage.
- Areas are in square kilometers. 
- Birth and death rates are per 1,000 people.

## Population statistics

### Find the minimum and maximum of `population` and `population_growth`:

In [3]:
%%sql
SELECT MIN(population) AS 'Lowest Population', MAX(population) AS 'Highest Population', 
       MIN(population_growth) AS 'Lowest Population Growth', MAX(population_growth) AS 'Highest Population Growth'
    FROM facts
    ;

 * sqlite:///factbook.db
Done.


Lowest Population,Highest Population,Lowest Population Growth,Highest Population Growth
0,7256490011,0.0,4.02


There are abnormalities here: one country has a population of zero, and another has a population of over 7.2 billion, equal to most of the world's population.

Return countries matching the value of the lowest population:

In [4]:
%%sql
SELECT name AS Country
    FROM facts
  WHERE population = (SELECT MIN(population)
                            FROM facts)
  ;

 * sqlite:///factbook.db
Done.


Country
Antarctica


Return countries matching the value of the highest population:

In [5]:
%%sql
SELECT name AS Country
    FROM facts
  WHERE population = (SELECT MAX(population)
                            FROM facts)
  ;

 * sqlite:///factbook.db
Done.


Country
World


The abberant rows are `Antarctica`, with a population of zero, and `World`. The row representing the whole world should be excluded from the summary statistics:

In [6]:
%%sql
SELECT MIN(population) AS 'Lowest Population', MAX(population) AS 'Highest Population',
       MIN(population_growth) AS 'Lowest Population Growth', MAX(population_growth) AS 'Highest Population Growth'
    FROM facts
  WHERE population < (SELECT MAX(population)
                          FROM facts)
    ;

 * sqlite:///factbook.db
Done.


Lowest Population,Highest Population,Lowest Population Growth,Highest Population Growth
0,1367485388,0.0,4.02


The highest population value is now over 1.3 billion. 

### Which country has the highest population?

In [16]:
%%sql
SELECT name, population
    FROM facts
    WHERE population < (SELECT MAX(population) 
                          FROM facts)
    ORDER BY population DESC
    LIMIT 1
    ;

 * sqlite:///factbook.db
Done.


name,population
China,1367485388


China has the highest population.

### Which countries will add the most people to their populations next year?

In [8]:
%%sql
SELECT name AS Country, population AS Population, population_growth AS 'Growth Rate', 
        CAST(ROUND(population*population_growth/100, 0) AS INT) AS 'Additional Population'
    FROM facts
    WHERE population < (SELECT MAX(population)
                          FROM facts) 
    ORDER BY CAST(ROUND(population*population_growth/100, 0) AS INT) DESC
    LIMIT 10
    ;

 * sqlite:///factbook.db
Done.


Country,Population,Growth Rate,Additional Population
India,1251695584,1.22,15270686
China,1367485388,0.45,6153684
Nigeria,181562056,2.45,4448270
Pakistan,199085847,1.46,2906653
Ethiopia,99465819,2.89,2874562
Bangladesh,168957745,1.6,2703324
United States,321368864,0.78,2506677
Indonesia,255993674,0.92,2355142
"Congo, Democratic Republic of the",79375136,2.45,1944691
Philippines,100998376,1.61,1626074


India is set to add the most people to its population next year: over 15 million, more than twice the growth of any other country. China has the lowest population growth rate (0.45), but is still second due to having the highest population. 

### Which countries have a higher death rate than birth rate?

In [9]:
%%sql
SELECT name AS Country, birth_rate AS 'Birth Rate', death_rate AS 'Death Rate', 
       ROUND(death_rate - birth_rate, 2) AS 'Death Rate - Birth Rate'
    FROM facts
    WHERE death_rate > birth_rate
    ORDER BY death_rate - birth_rate DESC
    ;

 * sqlite:///factbook.db
Done.


Country,Birth Rate,Death Rate,Death Rate - Birth Rate
Bulgaria,8.92,14.44,5.52
Serbia,9.08,13.66,4.58
Latvia,10.0,14.31,4.31
Lithuania,10.1,14.27,4.17
Ukraine,10.72,14.46,3.74
Hungary,9.16,12.73,3.57
Germany,8.47,11.42,2.95
Slovenia,8.42,11.37,2.95
Romania,9.14,11.9,2.76
Croatia,9.45,12.18,2.73


Twenty-four countries have a higher death rate than birth rate. Many of the countries are in Central and Eastern Europe. Bulgaria has the largest difference between death rate and birth rate. 

## Identifying densely-populated countries

Calculate the average values for `population` and `area`:

In [10]:
%%sql
SELECT ROUND(AVG(population), 0) AS 'Average Population', ROUND(AVG(area), 1) AS 'Average Area'
    FROM facts
    ;

 * sqlite:///factbook.db
Done.


Average Population,Average Area
62094928.0,555093.5


### Identify countries that have above-average values for `population` and below-average values for `area`:

In [11]:
%%sql
SELECT name, area, population
    FROM facts
  WHERE population > (SELECT AVG(population)
                          FROM facts)
        AND area < (SELECT AVG(area)
                           FROM facts)
    ;

 * sqlite:///factbook.db
Done.


name,area,population
Bangladesh,148460,168957745
Germany,357022,80854408
Japan,377915,126919659
Philippines,300000,100998376
Thailand,513120,67976405
United Kingdom,243610,64088222
Vietnam,331210,94348835


Seven countries have above-average population and below-average population.

### Calculate population density (`population`/`area`):

In [12]:
%%sql
SELECT name, population/area AS 'People/sq km'
    FROM facts
    ORDER BY population/area DESC
    LIMIT 10
    ;

 * sqlite:///factbook.db
Done.


name,People/sq km
Macau,21168
Monaco,15267
Singapore,8141
Hong Kong,6445
Gaza Strip,5191
Gibraltar,4876
Bahrain,1771
Maldives,1319
Malta,1310
Bermuda,1299


Macau, a city and special administrative region in China, is classified as a country here. It is indeed the most densely populated region in the world. The first list of countries with above-average population and below-average area was not helpful in identifying the countries with the true highest population density. Many of the highly dense countries from the list are small and would not have an above-average population, just a very dense one. 

## Water to land ratio

### Which countries have more water than land?

In [13]:
%%sql
SELECT name AS Country, area_land, area_water
    FROM facts
    WHERE area_water > area_land
    ;

 * sqlite:///factbook.db
Done.


Country,area_land,area_water
British Indian Ocean Territory,60,54340
Virgin Islands,346,1564


Only two countries, British Indian Ocean Territory and the Virgin Islands, have more water than land. 

### Which countries have the highest ratios of water to land? 

In [14]:
%%sql
SELECT name AS Country, ROUND(CAST(area_water AS FLOAT)/area_land, 2) AS 'Water:Land Ratio'
    FROM facts
    WHERE ROUND(CAST(area_water AS FLOAT)/area_land, 2) > 0.1
    ORDER BY CAST(area_water AS FLOAT)/area_land DESC
    ;

 * sqlite:///factbook.db
Done.


Country,Water:Land Ratio
British Indian Ocean Territory,905.67
Virgin Islands,4.52
Puerto Rico,0.55
"Bahamas, The",0.39
Guinea-Bissau,0.28
Malawi,0.26
Netherlands,0.23
Uganda,0.22
Eritrea,0.16
Liberia,0.16


British Indian Ocean Territory has the highest water to land ratio by far (over 900 times as much water). The Virgin Islands have about 4.5 times as much water as land. The next countries are other island nations such as Puerto Rico and The Bahamas. 